In [11]:
from IPython.core.display import HTML
HTML(open('../css/custom.css', 'r').read())

In [12]:
def print_stats(yosys_log):
    stat_start_line = yosys_log.grep(r'^2\.27\. ')
    stat_end_line = yosys_log.grep(r'^2\.28\. ')
    start_index = yosys_log.index(stat_start_line[0])
    end_index = yosys_log.index(stat_end_line[0])
    print('\n'.join(yosys_log[start_index+2:end_index-1]))


In [13]:
from pygmyhdl import *

@chunk
def ram(clk_i, en_i, wr_i, addr_i, data_i, data_o):
    # Create an array of words to act as RAM locations for storing data.
    # The number of bits in each word is set by the width of the data input bus.
    # The number of words is determined by the width of the address bus so,
    # for example, a 4-bit address would create 2**4 = 16 RAM locations.
    mem = [Bus(len(data_i)) for _ in range(2**len(addr_i))]
    
    # Perform read/write operations on the rising edge of the clock.
    @seq_logic(clk_i.posedge)
    def logic():
        if en_i:
            # The read/write operations only get executed if the enable input is high.
            if wr_i:
                # If the write-control is high, write the value on the input data bus
                # into the array of words at the given address value.
                mem[addr_i.val].next = data_i
            else:
                # If the write-control is low, read data from the word at the
                # given address value and send it to the output data bus.
                data_o.next = mem[addr_i.val]


In [14]:
@chunk
def simpler_ram(clk_i,wr_i, addr_i, data_i, data_o):

    mem = [Bus(len(data_i)) for _ in range(2**len(addr_i))]
    
    @seq_logic(clk_i.posedge)
    def logic():
        if wr_i:
            mem[addr_i.val].next = data_i
        data_o.next = mem[addr_i.val]  # RAM address is always read out!


In [15]:
@chunk
def dualport_ram(clk_i, wr_i, wr_addr_i, rd_addr_i, data_i, data_o):
    
    mem = [Bus(len(data_i)) for _ in range(2**len(wr_addr_i))]
    
    @seq_logic(clk_i.posedge)
    def logic():
        if wr_i:
            mem[wr_addr_i.val].next = data_i
        data_o.next = mem[rd_addr_i.val]  # Read from a different location than write.

In [18]:
@chunk
def gen_reset(clk_i, reset_o):


    cntr = Bus(1)  # Reset counter.
    
    @seq_logic(clk_i.posedge)
    def logic():
        if cntr < 1:
            # Generate a reset while the counter is less than some threshold
            # and increment the counter.
            cntr.next = cntr.next + 1
            reset_o.next = 1
        else:
            # Release the reset once the counter passes the threshold and
            # stop incrementing the counter.
            reset_o.next = 0

@chunk
def sample_en(clk_i, do_sample_o, frq_in=12e6, frq_sample=100):
    # Compute the width of the counter and when it should roll-over based
    # on the master clock frequency and the desired sampling frequency.
    from math import ceil, log2
    rollover = int(ceil(frq_in / frq_sample)) - 1
    cntr = Bus(int(ceil(log2(frq_in/frq_sample))))
    
    # Sequential logic for generating the sampling pulse.
    @seq_logic(clk_i.posedge)
    def counter():
        cntr.next = cntr + 1         # Increment the counter.
        do_sample_o.next = 0         # Clear the sampling pulse output except...
        if cntr == rollover:
            do_sample_o.next = 1     # ...when the counter rolls over.
            cntr.next = 0 

@chunk
def record_play(clk_i, button_a, button_b, leds_o):
    
    # Instantiate the reset generator.
    reset = Wire()
    gen_reset(clk_i, reset)
    
    # Instantiate the sampling pulse generator.
    do_sample = Wire()
    sample_en(clk_i, do_sample)
    
    # Instantiate a RAM for holding the samples.
    wr = Wire()
    addr = Bus(11)
    end_addr = Bus(len(addr)) # Holds the last address of the recorded samples.
    data_i = Bus(1)
    data_o = Bus(1)
    simpler_ram(clk_i, wr, addr, data_i, data_o)
    
    # States of the record/playback controller.
    state = Bus(3)         # Holds the current state of the controller.
    INIT = 0               # Initialize. The reset pulse sends us here.
    WAITING_TO_RECORD = 1  # Getting read to record samples.
    RECORDING = 2          # Actually storing samples in RAM.
    WAITING_TO_PLAY = 3    # Getting ready to play back samples.
    PLAYING = 4            # Actually playing back samples.

    # Sequential logic for the record/playback controller.
    @seq_logic(clk_i.posedge)
    def fsm():
        
        wr.next = 0        # Keep the RAM write-control off by default.
        
        if reset:  # Initialize the controller using the pulse from the reset generator.
            state.next = INIT  # Go to the INIT state after the reset is released.
            
        elif do_sample:  # Process a sample whenever the sampling pulse arrives.
        
            if state == INIT:  # Initialize the controller.
                leds_o.next = 0b10101  # Light LEDs to indicate the INIT state.
                if button_a == 1:
                    # Get ready to start recording when button A is pressed.
                    state.next = WAITING_TO_RECORD  # Go to record setup state.
                    
            elif state == WAITING_TO_RECORD:  # Setup for recording.
                leds_o.next = 0b11010  # Light LEDs to indicate this state.
                if button_a == 0:
                    # Start recording once button A is released.
                    addr.next = 0           # Start recording from beginning of RAM.
                    data_i.next = button_b  # Record the state of button B.
                    wr.next = 1             # Write button B state to RAM.
                    state.next = RECORDING  # Go to recording state.
                    
            elif state == RECORDING:  # Record samples of button B to RAM.
                addr.next = addr + 1    # Next location for storing sample.
                data_i.next = button_b  # Sample state of button B.
                wr.next = 1             # Write button B state to RAM.
                # For feedback to the user, display the state of button B on the LEDs.
                leds_o.next = concat(1,button_b, button_b, button_b, button_b)
                if button_a == 1:
                    # If button A pressed, then get ready to play back the stored samples.
                    end_addr.next = addr+1  # Store the last sample address.
                    state.next = WAITING_TO_PLAY  # Go to playback setup state.
                    
            elif state == WAITING_TO_PLAY:  # Setup for playback.
                leds_o.next = 0b10000  # Light LEDs to indicate this state.
                if button_a == 0:
                    # Start playback once button A is released.
                    addr.next = 0         # Start playback from beginning of RAM.
                    state.next = PLAYING  # Go to playback state.
                    
            elif state == PLAYING:  # Show recorded state of button B on the LEDs.
                leds_o.next = concat(1,data_o[0],data_o[0],data_o[0],data_o[0])
                addr.next = addr + 1  # Advance to the next sample.
                if addr == end_addr:
                    # Loop back to the start of RAM if this is the last sample.
                    addr.next = 0
                if button_a == 1:
                    # Record a new sample if button A is pressed.
                    state.next = WAITING_TO_RECORD



In [19]:
toVerilog(record_play, clk_i=Wire(), button_a=Wire(), button_b=Wire(), leds_o=Bus(5))
toVHDL(record_play, clk_i=Wire(), button_a=Wire(), button_b=Wire(), leds_o=Bus(5))

c:\users\kuban\anaconda3\envs\pygmyhdl\lib\site-packages\ipykernel_launcher.py:1: UserWarning: 
    toVerilog(): Deprecated usage: See http://dev.myhdl.org/meps/mep-114.html
  """Entry point for launching an IPython kernel.
c:\users\kuban\anaconda3\envs\pygmyhdl\lib\site-packages\ipykernel_launcher.py:2: UserWarning: 
    toVHDL(): Deprecated usage: See http://dev.myhdl.org/meps/mep-114.html
  


[[<myhdl._always_seq._AlwaysSeq at 0x23b3e2fe908>],